## ALGORITMO NAIVE-BAYES MULTINOMIAL - DETECCIÓN DE SPAM

Importamos librerías necesarias.  
Cargamos dataset usando un URL y a través de un REQUESTS

In [1]:
# pip install requests
import requests 
import zipfile
import pandas as pd

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
data_file = 'SMSSpamCollection'

# Make request
resp = requests.get(url)

# Get filename
filename = url.split('/')[-1]

# Download zipfile
with open(filename, 'wb') as f:
  f.write(resp.content)

# Extract Zip
with zipfile.ZipFile(filename, 'r') as zip:
  zip.extractall('')

# Read Dataset
data = pd.read_table(data_file, 
                     header = 0,
                     names = ['type', 'message']
                     )

# Show dataset
data.head()

,type,message
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


## Transformación de datos
Una vez tenemos los datos, lo primero de todo tenemos que procesarlos. Si bien el procesamiento de datos es un paso fundamental en todo proyecto de machine learning, en los proyectos de NLP (como este) procesar los datos resulta aún mucho más importante.

Así pues, lo primero de todo vamos seguir los siguientes procesos:

- **Tokenización**: consiste en separar los mensajes en palabras para así poder tratar cada una de las palabras. Esto lo podemos hacer gracias a la función word_tokenize.
- **Eliminación de stop words**: eliminación de palabras que no aportan valor (preposiciones, conjunciones, etc.). Esto se realiza puesto que aumentaría mucho el tamaño de nuestro dataset (ya de por sí grande) y no aportaría ningún valor, solo ruido. Existen dos grandes fuentes de stpwords en Python, las de Sklearn y las de la librería NLTK.

In [2]:
# Sklearn
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words
#text.text.ENGLISH_STOP_WORDS
# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jordi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

**Stemming** o **lemmatization**: el objetivo de este paso es que dos palabras que signifiquen lo mismo, pero no estén igual escritas, pasen a estar igual escritas. Al fin y al cabo, para el modelo la palabra «bueno» y la palabra «buena» son diferentes. Para hacer esto hay dos grandes técnicas:

- Stemming: el stemming consiste en la eliminación de las terminaciones de las palabras para quedarnos únicamente con la raíz. Siguiendo el caso anterior, en ambos casos (bueno, buena) nos quedaríamos con «buen».
- Lemmatization: es más utilizado en proyectos de NLP en inglés, ya que en este idioma bueno (good), mejor (better) y el mejor (best) son palabras completamente diferentes. En estos casos el stemming no funcionaría. Así pues, la lematización convertiría todas esas palabras a su base (good), de tal forma que pasen a significar lo mismo.  

Para realizar todos estos procesos usaremos el paquete **nltk**, es decir, el Natural Language Toolkit, el cual incluye muchas funcionalidades sobre NLP para Python y que, sin duda, serán muy utiles para usar Naive Bayes en Python.

In [3]:
nltk.download('punkt')

from nltk.stem.porter import *
stop = stopwords.words('english')

# Tokenize
data['tokens'] = data.apply(lambda x: nltk.word_tokenize(x['message']), axis = 1)

# Remove stop words
data['tokens'] = data['tokens'].apply(lambda x: [item for item in x if item not in stop])

# Apply Porter stemming
stemmer = PorterStemmer()
data['tokens'] = data['tokens'].apply(lambda x: [stemmer.stem(item) for item in x])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jordi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Una vez realizada la limpieza de nuestros datos, esto no acaba aquí. Y es que, actualmente únicamente disponemos de una columna que cuenta con un vector de palabras.

Naive Bayes admite dos cuestiones:

- Una matriz TF, es decir, una matriz en la que aparece, para cada documento, cuántas veces ha aparecido cada una de las palabras que hay en en todos los documentos.
- Una matriz de apariciones. Es similar a una matriz TF, pero en este caso, en vez de indicar el número de apariciones, simplemente indica si esa palabra aparecía o no.  

El uso de cada uno de ellas dependerá mucho del contexto. En el caso de SMS's, al ser mensajes muy cortos es poco probable que las palabras se repitan, por lo que seguramente ambos enfoques devuelvan el mismo resultado.

Sin embargo, en textos más largos seguramente sea más interesante aplicar una matriz TF que una matriz de apariciones.

Dicho esto, para poder llegar a una matriz TF vamos a hacer lo siguiente:

- Destokenizar los valores, de tal forma que la columna «tokens» no contenga listas, sino texto. Esto es necesario para que el tercer paso funcione correctamente.
- Hacer un split entre train y test. Es muy importante realizar el proceso de train y test antes de llegar a la matriz TF. Sino, tendremos problemas de data leakage y puede afectar a nuestro resultaod (incluso es nos permite comprobar que nuestro pipeline de datos es correcto).
- Aplicar la función CountVectorizer del módulo feature_extraction.text de Sklearn a nuestros datos de train y test. Esta función nos permite crear la matriz TF o, si indicamos el parámetro binary = True, creará una matriz de apariciones.  

Dicho esto, veamos cómo poder aplicar la matriz TF para poder entrenar el modelo de NLP con Naive Bayes en Python:

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# Unify the strings once again
data['tokens'] = data['tokens'].apply(lambda x: ' '.join(x))

# Make split
x_train, x_test, y_train, y_test = train_test_split(
    data['tokens'], 
    data['type'], 
    test_size= 0.2
    )

# Create vectorizer
vectorizer = CountVectorizer(
    strip_accents = 'ascii', 
    lowercase = True
    )

# Fit vectorizer & transform it
vectorizer_fit = vectorizer.fit(x_train)
x_train_transformed = vectorizer_fit.transform(x_train)
x_test_transformed = vectorizer_fit.transform(x_test)

## Construcción y entrenamiento del modelo

In [5]:
# Build the model
from sklearn.naive_bayes import MultinomialNB

# Train the model
naive_bayes = MultinomialNB()
naive_bayes_fit = naive_bayes.fit(x_train_transformed, y_train)

from sklearn.metrics import confusion_matrix, balanced_accuracy_score

# Make predictions
train_predict = naive_bayes_fit.predict(x_train_transformed)
test_predict = naive_bayes_fit.predict(x_test_transformed)

def get_scores(y_real, predict):
  ba_train = balanced_accuracy_score(y_real, predict)
  cm_train = confusion_matrix(y_real, predict)

  return ba_train, cm_train 

def print_scores(scores):
  return f"Balanced Accuracy: {scores[0]}\nConfussion Matrix:\n {scores[1]}"

train_scores = get_scores(y_train, train_predict)
test_scores = get_scores(y_test, test_predict)


print("## Train Scores")
print(print_scores(train_scores))
print("\n\n## Test Scores")
print(print_scores(test_scores))

## Train Scores
Balanced Accuracy: 0.9841222942124109
Confussion Matrix:
 [[3857   11]
 [  17  571]]


## Test Scores
Balanced Accuracy: 0.9696060629983422
Confussion Matrix:
 [[952   4]
 [  9 150]]
